In [1]:
import pandas as pd
import numpy as np

In [2]:
%store -r

In [8]:
# Import data and create df

data = pd.read_csv(
    "/Users/Stacy/Documents/WebDevelopment/projects/SPD/sexualAssault_datasets/2010-2018_sexcrimes_violentcrimes.csv",
    index_col="CAD Event Number")
data = pd.DataFrame(data)

In [9]:
# Rename columns to be pythonic

data.columns = ['event_clearing', 'call_type', 'priority', 
                'initial_call_type', 'final_call_type', 'original_time_queued',
               'arrived_time', 'precinct', 'sector', 'beat']

In [18]:
# Replace final_call_type values to english

data['final_call_type'] = data['final_call_type'].replace("--HOMICIDE", "Homicide")
data['final_call_type'] = data['final_call_type'].replace("--ASSAULTS - FIREARM INVOLVED", "Assault w/ Firearm")
data['final_call_type'] = data['final_call_type'].replace("--ROBBERY - ARMED", "Armed Robbery")
data['final_call_type'] = data['final_call_type'].replace("--KIDNAP - GENERAL", "Kidnap")
data['final_call_type'] = data['final_call_type'].replace("--DRIVE BY SHOOTING - NO INJURIES", "Drive-by Shooting")
data['final_call_type'] = data['final_call_type'].replace("--SEX OFFENSES (NON-RAPE) - LEWD CONDUCT", "Lewd Conduct")
data['final_call_type'] = data['final_call_type'].replace("--RAPE - KNOWN SUSPECT (ACQUAINTANCE)", "Acquaintance Rape")
data['final_call_type'] = data['final_call_type'].replace("--RAPE - UNKNOWN SUSPECT (STRANGER)", "Stranger Rape")
data['final_call_type'] = data['final_call_type'].replace("--SEX OFFENSES (RAPE) - MOLESTING", "Molesting")
data['final_call_type'] = data['final_call_type'].replace("--COMMERCIAL SEXUAL EXPLOITATION OF MINORS (CSEC)", "CSEC")

In [27]:
# Clean up precinct names

# Replace precinct names with lowercase
data['precinct'] = data['precinct'].str.title()


In [11]:
# ID all calls cancelled by dispatch
# cancelled_calls = data[data.event_clearing.str.contains("CANCELLED")]

# Delete rows with cxl'd calls

data = data[data.event_clearing != 'RESPONDING UNIT(S) CANCELLED BY RADIO']
data = data[data.event_clearing != 'DUPLICATED OR CANCELLED BY RADIO']

### Clean up times and dates

In [12]:
# Change original_time_queued to datetime
data['original_time_queued'] = pd.to_datetime(data.original_time_queued)

# Change arrived_time to datetime
data['arrived_time'] = pd.to_datetime(data.arrived_time, format='%b %d %Y %I:%M:%S:%f%p')

# Add time delta between original_time_queued and arrived_time
data['time_delta'] = ((data.arrived_time - data.original_time_queued)/np.timedelta64(1, 's')).astype(int)

In [14]:
# Add columns for arrived year and month
data['year'] = data.arrived_time.dt.year
data['month'] = data.arrived_time.dt.month

# Where arrive_time year = 1900, change data.year and data.month to original_time_queued year
mask = data.year < 1901
column_name_year = 'year'
column_name_month = 'month'
data.loc[mask, column_name_year] = data.original_time_queued.dt.year
data.loc[mask, column_name_month] = data.original_time_queued.dt.month

### Define terms

In [17]:
# Define TRU initial call types
TRU_call_types = [
    'TRU - ASLT - WITH OR W/O WPNS (NO SHOOTINGS)',
    'TRU - RAPE',
    'TRU - LEWD CONDUCT',
    'TRU - CHILD ABUSED, NEGLECTED',
    'TRU - THREATS',
    'TRU - ROBBERY',
    'TRU - ASLT - MOLESTED ADULT (GROPED, FONDLED, ETC)',
    'TRU - HARASSMENT',
    'TRU - SEX IN PUBLIC',
    'TRU - THEFT',
    'TRU - FOLLOW UP',
    'TRU - MVC - HIT AND RUN',
    'TRU - LURING',
    'TRU - FORGERY/CHKS/BUNCO/SCAMS/ID THEFT',
    'TRU - SUSPICIOUS CIRCUMSTANCES']

In [19]:
# Define crime_type terms

# Define terms that ID rape CFS
rape_terms = 'Stranger|Acquaintance'
rape_call_types = [
    'Acquaintance Rape',
    'Stranger Rape']

# Define terms that ID sex crimes
sex_crime_terms = 'CSEC|Lewd|Molesting|Stranger|Acquaintance'
sex_crime_call_types = [
    'Lewd Conduct',
    'Acquaintance Rape',
    'Stranger Rape',
    'Molesting',
    'CSEC']

# Define terms that ID other violent crimes
violent_crime_terms = 'Kidnap|Homicide|Shooting|Assault|Robbery'

In [20]:
# Define clearing codes for is_arrest

arrest_cc = ['PHYSICAL ARREST MADE']

### Add columns for is_arrest, is_TRU, crime_type

In [22]:
# Add column identifying if there was an arrest
data['is_arrest'] = np.where(data['event_clearing'].isin(arrest_cc), 'yes', 'no')

# Add column identifying if this was A TRU call
data['is_TRU'] = np.where(data['initial_call_type'].isin(TRU_call_types), 'yes', 'no')

In [24]:
# Add crime_type for each call type as a violent crime, sex crime or rape
is_rape = data['final_call_type'].str.contains(rape_terms)
is_vc = data['final_call_type'].str.contains(violent_crime_terms)
is_sex_crime = data['final_call_type'].str.contains(sex_crime_terms)
is_lewd_conduct = data['final_call_type'].str.contains("Lewd")
is_tru_mask = data['is_TRU'] == "yes"

data['crime_type'] = np.where(data['final_call_type'].isin(rape_call_types), 'rape',
                                (np.where(data['final_call_type'].isin(sex_crime_call_types),
                                          'other sex crime', 'violent crime')))

In [31]:
data.head()

,event_clearing,call_type,priority,initial_call_type,final_call_type,original_time_queued,arrived_time,precinct,sector,beat,time_delta,year,month,is_arrest,is_TRU,crime_type
CAD Event Number,,,,,,,,,,,,,,,,
2010000066820,ASSISTANCE RENDERED,911,2,MENTAL - PERSON OR PICKUP/TRANSPORT,Lewd Conduct,2010-02-27 22:54:25,2010-02-27 22:55:45,North,UNION,U1,80,2010,2,no,no,other sex crime
2010000000327,PHYSICAL ARREST MADE,911,1,ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS),Assault w/ Firearm,2010-01-01 03:13:06,2010-01-01 03:14:51,South,ROBERT,R3,105,2010,1,yes,no,violent crime
2010000000451,REPORT WRITTEN (NO ARREST),911,2,THEFT OF SERVICES,Armed Robbery,2010-01-01 05:46:54,2010-01-01 06:39:35,North,UNION,U1,3161,2010,1,no,no,violent crime
2010000000466,REPORT WRITTEN (NO ARREST),"TELEPHONE OTHER, NOT 911",3,RAPE,Acquaintance Rape,2010-01-01 06:28:49,2010-01-01 06:40:09,East,GEORGE,G1,680,2010,1,no,no,rape
2010000000776,REPORT WRITTEN (NO ARREST),911,3,RAPE,Acquaintance Rape,2010-01-01 15:05:57,2010-01-01 15:12:26,Southwest,FRANK,F3,389,2010,1,no,no,rape
